In [1]:
import geopandas as gpd
import pandas as pd
import pandas
import math
from shapely.geometry import Point, LineString, Polygon, MultiPoint, MultiLineString, MultiPolygon
from geographiclib.geodesic import Geodesic
from shapely.ops import split
from shapely import ops
import shapely
from pyproj import Proj, Transformer

In [2]:
buildings= gpd.read_file('../tests/data/buildings.geojson')
road= gpd.read_file('../tests/data/road.geojson')

In [3]:
# for d in buildings.columns:
#     print(d)
road['end'] = None
road['start'] = None
road['start_point_x'] = None
road['start_point_y'] = None
road['end_point_x'] = None
road['end_point_y'] = None
road['length'] = None
road['bearing'] = None
road['intersected'] = None
road['mother_road']='None'

buildings['road_direction'] = 'None'
buildings['from_mother_road_direction'] = 'None'

buildings['distance2_intersecting_road'] = 0
buildings['road_length_d_code'] = ''

# print(buildings)
# print(road)

In [4]:
class bdcPoint:
    	
	def __init__(self):
		
		self.x = 0
		self.y = 0

# Constant integers for directions
RIGHT = 1
LEFT = -1
ZERO = 0

def directionOfPoint(A, B, P):
    """[Provides direction of the point in respect to line]

    Args:
        A ([type]): [Starting Point of Line]
        B ([type]): [Ending Point of Line]
        P ([type]): [Point of Interest]

    Returns:
        [Direction(int)]: [Left,Right and Zero : -1,1,0 ( Left direction, Right Direction , On Line)]
    """
    global RIGHT, LEFT, ZERO
	
	# Subtracting co-ordinates of
	# point A from B and P, to
	# make A as origin
 
    B.x -= A.x
    B.y -= A.y
    P.x -= A.x
    P.y -= A.y

    # Determining cross Product
    """"The Cross-Product has an interesting Property which will be used to determine direction of a point from a line segment. That is, the cross-product of two points is positive if and only if the angle of those point at origin (0, 0) is in counter-clockwise. And conversely the cross-product is negative if and only if the angle of those point at origin is in clockwise direction."""
    cross_product = B.x * P.y - B.y * P.x
    # Return RIGHT if cross product is positive
    if (cross_product > 0):
        return RIGHT
        
    # Return LEFT if cross product is negative
    if (cross_product < 0):
        return LEFT

    # Return ZERO if cross product is zero
    return ZERO

def testLeftRightofLine(start,end,poi):
    d=(poi.x-start.x)*(end.y-start.y)-(poi.y-start.y)*(end.x-start.x)
    if d>0:
        #left
        return -1
    if d<0:
        #right
        return 1
    else:
        return 0

def findMotherRoad2AssociatedRoadDirection(mother_road,associated_road):
    left_buffered_line=mother_road.buffer(0.0001,single_sided=True)
#     right_buffered_line=direction_updated_roi.buffer(0.0007,single_sided=True)
    if left_buffered_line.intersects(associated_road.buffer(0.00001)) == True:
        direction=-1
#         print("left")
    else:
        direction=1
#         print("right")
    return direction

def addDirection_BuildingCode(direction,building_length,road_length,index,road_index):
    #right-even, left-odd number 
    # print(length)
    building_length=int(building_length)
    if road_length < 60:
        associated_road=road.iloc[road_index]

        if associated_road.mother_road == 'start':
            mother_road_code=associated_road.start
        else :
            mother_road_code=associated_road.end

        mother_road= road.loc[road['NEW'] == mother_road_code]

        mr_geom= mother_road.geometry
        d = {'col1': ['name1'], 'geometry': [Point(associated_road.start_point_x, associated_road.start_point_y)]}
        motherroad_start_point = {'col1': ['name1'], 'geometry': [Point(mother_road.start_point_x, mother_road.start_point_y)]}
        
        gdf = gpd.GeoDataFrame(d, crs=4326)
        mother_df = gpd.GeoDataFrame(motherroad_start_point, crs=4326)
        
        sp_point=gdf.iloc[0]
        mother_st_p=mother_df.iloc[0]
        for line in mr_geom:
            # print(line)
            for l in line:
                mother_line=l
                break

        result=split(mother_line,sp_point.geometry)
        for r  in result:
            splitted_line=r
            splitted_line.srid = 4326
            if splitted_line.contains(mother_st_p.geometry):
                break
            else:
                continue
        my_transformer = Transformer.from_crs('EPSG:4326', 'EPSG:32644', always_xy=True)
        geom_transformed = ops.transform(my_transformer.transform, splitted_line)
#         mother_road_direction=findMotherRoad2AssociatedRoadDirection(mother_road.geometry,associated_road.geometry)
        buildings.at[index,'road_length_d_code']=str(int(geom_transformed.length))+"/"
        
#         if (mother_road_direction == 1):
#             # print("Right Direction")
#             buildings.at[index,'from_mother_road_direction']="right"
#             buildings.at[index,'road_length_d_code']=str(int(round_up_to_even(geom_transformed.length)))+"/"
#         if (mother_road_direction == -1):
#             # print("left Direction")
#             buildings.at[index,'from_mother_road_direction']="left"
#             buildings.at[index,'road_length_d_code']=str(int(round_up_to_odd(geom_transformed.length)))+"/"
            
        if (direction == 1):
            # print("Right Direction")
            buildings.at[index,'road_direction']="right"
            buildings.at[index,'road_length_d_code']+=str(int(round_up_to_even(building_length)))

        elif (direction == -1):
            # print("Left Direction")
            buildings.at[index,'road_direction']="left"
            buildings.at[index,'road_length_d_code']+=str(int(round_up_to_odd(building_length)))
        else:
            print("On  Line")
            buildings.at[index,'road_direction']="on_line"
    else :
        if (direction == 1):
            # print("Right Direction")
            buildings.at[index,'road_direction']="right"
            buildings.at[index,'road_length_d_code']=str(int(round_up_to_even(building_length)))

        elif (direction == -1):
            # print("Left Direction")
            buildings.at[index,'road_direction']="left"
            buildings.at[index,'road_length_d_code']=str(int(round_up_to_odd(building_length)))
        else:
            print("On  Line")
            buildings.at[index,'road_direction']="on_line"
    
    
#     print(buildings.at[index,'road_length_d_code'])
            
def getStartEndPoint(line,type):
    if len(line)==1 or type=="road" :
       
        df_line = line.geometry.boundary
        # print(len(df_line))
        # print(line_bound)
        # df_line=line_bound.explode(index_parts=True)
        start_point,end_point=df_line[0],df_line[1]

        return start_point,end_point
    else:
        raise ValueError ("Line contains more than one code")

def getLineLength(line):
    projected_line = line.to_crs(epsg=32644)
    length= projected_line.geometry.length
    float_length=length.tolist()
    return float_length

def round_up_to_even(f):
    return math.ceil(f / 2.) * 2

def round_up_to_odd(f):
    return math.ceil(f) // 2 * 2 + 1

def addStartEndPoint(start_point,end_point,index):
    
    road.at[index,'start_point_x']=float(start_point.x)
    road.at[index,'start_point_y']=float(start_point.y)
    
    road.at[index,'end_point_x']=float(end_point.x)
    road.at[index,'end_point_y']=float(end_point.y)

def ReviseStartEndDateWithBearing(bearing,index):
    # default direction is west to east , south to north 
    # print("ma revise garna aako dubai vettera")
    if bearing is None :
        bearing=0
    bearing=int(bearing)
    def assign_start():
        print("start")
        
        road.at[index,'mother_road']='start'

    def assign_end():
    # east_west/west_east direction , assign to east
        print("end")
        road.at[index,'mother_road']='end'

    
    if ((bearing >= 0) and (bearing<=45)) :
        #    North_South/south_north Direction, assign to south
        
        assign_start()
        
    if ((bearing > 45) and (bearing<135)) :
        
        # east_west/west_east direction , assign to east
        assign_end()
    
    if ((bearing >= 135) and (bearing<=225)) :
        #    North_South/south_north Direction, assign to south
        
        assign_start()
    if ((bearing > 225) and (bearing<315)) :
            
        # east_west/west_east direction , assign to east
        assign_end()
    if ((bearing >= 315) and (bearing<=360)) :
    #    North_South/south_north Direction, assign to south
        
        assign_start()      

def FindIntersectedFeatures(start_point,end_point,line_row,index_road):
    for index, row in road.iterrows():
        line=row.geometry        
        if str(row.NEW) != str(line_row.NEW):
            if line.distance(start_point) < 1e-8 :
                # print("start found "+ str(row.NEW))
                # print("mero start vetiyo guys ")
                road.at[index_road,'start']=str(row.NEW)
    
            if line.distance(end_point) < 1e-8:
                # print("end found "+ str(row.NEW))
                road.at[index_road,'end']=str(row.NEW)
                # print("mero end vetiyo guys ")


def scan_I_feature(line_row,index_road):
    
        if ((line_row.start!= None) and (line_row.end== None)) :
            road.at[index_road,'mother_road']='start'
        if ((line_row.start== None) and (line_row.end != None)) :
            road.at[index_road,'mother_road']='end'
    
        if ((line_row.start!= None) and (line_row.end != None)) :
        #both start and end exist which means line is in between roads 
            road.at[index_road,'mother_road']='None'
            ReviseStartEndDateWithBearing(line_row.bearing,index_road)

                    
def CalculateBearing(start_point,end_point,index):
    brng = Geodesic.WGS84.Inverse(start_point.y, start_point.x, end_point.y, end_point.x)['azi1']
    road.at[index,'bearing']=brng

def is_number(string):
    try:
        float(string)
        return True
    except ValueError:
        return False    
    
def AssignIntersectedPoint(start_point_assumed,end_point_assumed,row,index):
#     print(row)
    if  row.mother_road :
        mother_road_direction = row.mother_road
        mother_road_code=None
        if mother_road_direction == 'start':
            mother_road_code=row.start
        elif mother_road_direction == 'end' :
            mother_road_code=row.end
        mother_road= road.loc[road['NEW'] == mother_road_code].geometry
        current_road = row.geometry
        mother_road_geom=mother_road
        for l in mother_road:
            for single in l:
                mother_road_geom=single
            break
#         print(mother_road_geom)
        for l in current_road:
            road_geom=l
            break
#         print(road_geom)
        start_point=mother_road_geom.intersection(road_geom)
        print(start_point)
        st=str(type(start_point))
#         print(st)
#         geoseries=f"""<class 'geopandas.geoseries.GeoSeries'>"""
#         print(geoseries)
    
        if isinstance(start_point,pandas.core.series.Series) :
            print(type(start_point))
            print('not counted')
            
            road.at[index,'intersected']='No'
        else:
            print(start_point)
            try:
                x=start_point.x
                y=start_point.y
                road.at[index,'start_point_x']=float(x)
                road.at[index,'start_point_y']=float(y)              
            except:
                try:
                    for p in start_point:
                        x=p.x
                        y=p.y
                        break
                    road.at[index,'start_point_x']=x
                    road.at[index,'start_point_y']=y
                except:
                    try:
                        df_line = start_point.boundary
                        start=df_line[0]
                        road.at[index,'start_point_x']=start.x
                        road.at[index,'start_point_y']=start.y
                    except:
                        print('i didnt assigned')
                        pass
#                     road.at[index,'start_point_x']=0
#                     road.at[index,'start_point_y']=0
 

        
def get_building_projected_road_length(road_linked_wrt_point,row):
    road_code = row.NEW
#     print(road_linked_wrt_point)
    linked_road_geometry = road_linked_wrt_point.geometry
    building_geometry = row.geometry
    degree_length_for_building=linked_road_geometry.project(building_geometry)
    # 5m=0.00005 in degree 
    building_point_projected_on_road=linked_road_geometry.interpolate(degree_length_for_building) #end_always 
    for r in building_point_projected_on_road:
        building_point_projected_on_road_single_geom=r
        break
    try:
        projected_building_on_road = {'col1': ['name1'], 'geometry': [Point(building_point_projected_on_road_single_geom.x, building_point_projected_on_road_single_geom.y)]}
    except:
        print(projected_building_on_road)
        raise("found error")

    road_link_wrt_point_x=road_linked_wrt_point.start_point_x
    road_link_wrt_point_y=road_linked_wrt_point.start_point_y
    try:
        road_start_point = {'col1': ['name1'], 'geometry': [Point(road_link_wrt_point_x,road_link_wrt_point_y)]}

        projected_building_on_road_gdf = gpd.GeoDataFrame(projected_building_on_road, crs=4326)


        road_start_df = gpd.GeoDataFrame(road_start_point, crs=4326)


        splitting_point=projected_building_on_road_gdf.iloc[0]
        start_point=road_start_df.iloc[0]
#         print(start_point.geometry)
        for line in linked_road_geometry:
            # print(line)
            for l in line:
                mother_line=l
                break
        result=split(mother_line,splitting_point.geometry.buffer(0.0000001))

        for r  in result:
                splitted_line=r
                splitted_line.srid = 4326
                if splitted_line.buffer(0.00001).contains(start_point.geometry.buffer(0.0000001)) == True:

                    break
                else:
                    continue


        
        my_transformer = Transformer.from_crs('EPSG:4326', 'EPSG:32644', always_xy=True)
        geom_transformed = ops.transform(my_transformer.transform, splitted_line)

        clipped_length=geom_transformed.length
        return clipped_length ,splitted_line ,building_point_projected_on_road_single_geom,start_point.geometry,degree_length_for_building
    except:
        pass



    
                   


In [16]:
def CalculateTouchLine(index,row):
    
    # print(row)
    start_point,end_point=getStartEndPoint(row,"road")
    addStartEndPoint(start_point,end_point,index)
    CalculateBearing(start_point,end_point,index)
    FindIntersectedFeatures(start_point,end_point,row,index)
    scan_I_feature(row,index)
#     print(row)
    AssignIntersectedPoint(start_point,end_point,row,index)
    # break


In [23]:
for index, row in road.iterrows ():
    CalculateTouchLine(index,row)
    print(f"""i: {index}""")
    

GeoSeries([], dtype: geometry)
<class 'geopandas.geoseries.GeoSeries'>
not counted
i: 0
end
POINT (83.47182876258739 27.59323857200252)
POINT (83.47182876258739 27.59323857200252)
i: 1
POINT (83.47268607010614 27.60246986818993)
POINT (83.47268607010614 27.60246986818993)
i: 2
POINT (83.47040628242405 27.57787412609008)
POINT (83.47040628242405 27.57787412609008)
i: 3
POINT (83.47172783257349 27.59225408505802)
POINT (83.47172783257349 27.59225408505802)
i: 4
end
POINT (83.46736454413843 27.56698659255727)
POINT (83.46736454413843 27.56698659255727)
i: 5
POINT (83.47153400529015 27.59036561757927)
POINT (83.47153400529015 27.59036561757927)
i: 6
end
POINT (83.47187124206425 27.59364565192533)
POINT (83.47187124206425 27.59364565192533)
i: 7
end
POINT (83.47263962641767 27.60196558594311)
POINT (83.47263962641767 27.60196558594311)
i: 8
POINT (83.47213499163661 27.59644264511201)
POINT (83.47213499163661 27.59644264511201)
i: 9
POINT (83.47071132616685 27.58135272534804)
POINT (83.47071

LINESTRING EMPTY
LINESTRING EMPTY
i didnt assigned
i: 83
start
POINT (83.46549076978323 27.56908419288123)
POINT (83.46549076978323 27.56908419288123)
i: 84
POINT (83.46409408127687 27.60264427461362)
POINT (83.46409408127687 27.60264427461362)
i: 85
end
MULTIPOINT (83.46945272316469 27.58678523215281, 83.47021403794753 27.5861709277462)
MULTIPOINT (83.46945272316469 27.58678523215281, 83.47021403794753 27.5861709277462)
i: 86
start
POINT (83.46930570919119 27.5946145949876)
POINT (83.46930570919119 27.5946145949876)
i: 87
POINT (83.46806967652928 27.57928846131166)
POINT (83.46806967652928 27.57928846131166)
i: 88
start
POINT (83.45892516306589 27.56683897154136)
POINT (83.45892516306589 27.56683897154136)
i: 89
POINT (83.47150689522709 27.59260663009195)
POINT (83.47150689522709 27.59260663009195)
i: 90
POINT (83.4702157331696 27.59357258021043)
POINT (83.4702157331696 27.59357258021043)
i: 91
POINT (83.46215994622457 27.57253243552356)
POINT (83.46215994622457 27.57253243552356)
i: 

POINT (83.46079436976441 27.58428014900971)
POINT (83.46079436976441 27.58428014900971)
i: 169
POINT (83.43969330140482 27.59592745678674)
POINT (83.43969330140482 27.59592745678674)
i: 170
POINT (83.45864736968042 27.5734361417646)
POINT (83.45864736968042 27.5734361417646)
i: 171
POINT (83.43984491810852 27.59583754436721)
POINT (83.43984491810852 27.59583754436721)
i: 172
POINT (83.45459247316415 27.59034999995259)
POINT (83.45459247316415 27.59034999995259)
i: 173
end
POINT (83.46031340064297 27.5741794943882)
POINT (83.46031340064297 27.5741794943882)
i: 174
POINT (83.45914828396621 27.57330037741053)
POINT (83.45914828396621 27.57330037741053)
i: 175
POINT (83.45997992213574 27.58453232250872)
POINT (83.45997992213574 27.58453232250872)
i: 176
start
POINT (83.45543963812884 27.5938612392049)
POINT (83.45543963812884 27.5938612392049)
i: 177
start
POINT (83.43780754538199 27.59591046139872)
POINT (83.43780754538199 27.59591046139872)
i: 178
end
POINT (83.45892516306589 27.56683897

end
POINT (83.46270805783064 27.57742490393622)
POINT (83.46270805783064 27.57742490393622)
i: 255
start
POINT (83.46384818864306 27.58277196525404)
POINT (83.46384818864306 27.58277196525404)
i: 256
POINT (83.46717947625189 27.60665606482559)
POINT (83.46717947625189 27.60665606482559)
i: 257
end
POINT (83.46905658439391 27.60298940383797)
POINT (83.46905658439391 27.60298940383797)
i: 258
start
POINT (83.46759247821035 27.59867287745141)
POINT (83.46759247821035 27.59867287745141)
i: 259
end
POINT (83.46304003536937 27.5820388091438)
POINT (83.46304003536937 27.5820388091438)
i: 260
POINT (83.46439721306098 27.5898755032519)
POINT (83.46439721306098 27.5898755032519)
i: 261
POINT (83.46987896674096 27.58024431902015)
POINT (83.46987896674096 27.58024431902015)
i: 262
POINT (83.46165059270078 27.57643070251498)
POINT (83.46165059270078 27.57643070251498)
i: 263
POINT (83.46424754698677 27.57094090799654)
POINT (83.46424754698677 27.57094090799654)
i: 264
end
POINT (83.46641435104152 2

i: 337
POINT (83.46671072981638 27.59917494916465)
POINT (83.46671072981638 27.59917494916465)
i: 338
POINT (83.466986702974 27.58818580303245)
POINT (83.466986702974 27.58818580303245)
i: 339
POINT (83.46182581840685 27.57691377335203)
POINT (83.46182581840685 27.57691377335203)
i: 340
end
MULTIPOINT (83.46577314701318 27.56919417726954, 83.46717383480467 27.56974050462065)
MULTIPOINT (83.46577314701318 27.56919417726954, 83.46717383480467 27.56974050462065)
i: 341
POINT (83.46641435104152 27.59787895774775)
POINT (83.46641435104152 27.59787895774775)
i: 342
POINT (83.46549540668775 27.5693259864052)
POINT (83.46549540668775 27.5693259864052)
i: 343
end
POINT (83.4690080479831 27.59301898903516)
POINT (83.4690080479831 27.59301898903516)
i: 344
GeoSeries([], dtype: geometry)
<class 'geopandas.geoseries.GeoSeries'>
not counted
i: 345
POINT (83.46458372795672 27.57234261022336)
POINT (83.46458372795672 27.57234261022336)
i: 346
end
POINT (83.46898583113125 27.6002747399873)
POINT (83.46

...

end
POINT (83.43917845392258 27.58777460601328)
POINT (83.43917845392258 27.58777460601328)
i: 503
end
POINT (83.46222015132285 27.58936818939479)
POINT (83.46222015132285 27.58936818939479)
i: 504
LINESTRING EMPTY
LINESTRING EMPTY
i didnt assigned
i: 505
POINT (83.46244568961691 27.57812763148507)
POINT (83.46244568961691 27.57812763148507)
i: 506
POINT (83.46502471052202 27.58980113561398)
POINT (83.46502471052202 27.58980113561398)
i: 507
POINT (83.46830683938919 27.59550533570274)
POINT (83.46830683938919 27.59550533570274)
i: 508
start
POINT (83.46523515547813 27.58974606832646)
POINT (83.46523515547813 27.58974606832646)
i: 509
POINT (83.46798387035938 27.58000348867012)
POINT (83.46798387035938 27.58000348867012)
i: 510
POINT (83.46433043570113 27.58958547459002)
POINT (83.46433043570113 27.58958547459002)
i: 511
GeoSeries([], dtype: geometry)
<class 'geopandas.geoseries.GeoSeries'>
not counted
i: 512
POINT (83.4675750817247 27.57502455493932)
POINT (83.4675750817247 27.57502455

i: 587
POINT (83.45773009902945 27.57006130296623)
POINT (83.45773009902945 27.57006130296623)
i: 588
POINT (83.45056105344992 27.60037932664466)
POINT (83.45056105344992 27.60037932664466)
i: 589
end
POINT (83.46089897081089 27.59753677560178)
POINT (83.46089897081089 27.59753677560178)
i: 590
start
POINT (83.43722757799037 27.60148927081326)
POINT (83.43722757799037 27.60148927081326)
i: 591
POINT (83.45843145590953 27.60247824973407)
POINT (83.45843145590953 27.60247824973407)
i: 592
POINT (83.45953458595244 27.57441454659408)
POINT (83.45953458595244 27.57441454659408)
i: 593
POINT (83.46130014128858 27.6055477529265)
POINT (83.46130014128858 27.6055477529265)
i: 594
end
POINT (83.46213164366048 27.6061494920055)
POINT (83.46213164366048 27.6061494920055)
i: 595
POINT (83.45487302027374 27.60659482279289)
POINT (83.45487302027374 27.60659482279289)
i: 596
end
POINT (83.44047797787641 27.59921796014078)
POINT (83.44047797787641 27.59921796014078)
i: 597
POINT (83.43294214656288 27.5

POINT (83.42474947470856 27.58725461980492)
i: 676
POINT (83.43771639819329 27.59472556863017)
POINT (83.43771639819329 27.59472556863017)
i: 677
POINT (83.42290755703817 27.5783491947596)
POINT (83.42290755703817 27.5783491947596)
i: 678
POINT (83.43097077238332 27.56153676660853)
POINT (83.43097077238332 27.56153676660853)
i: 679
POINT (83.42183814441489 27.57595739831532)
POINT (83.42183814441489 27.57595739831532)
i: 680
POINT (83.45044484485379 27.57920657872069)
POINT (83.45044484485379 27.57920657872069)
i: 681
GeoSeries([], dtype: geometry)
<class 'geopandas.geoseries.GeoSeries'>
not counted
i: 682
end
MULTIPOINT (83.42559174915993 27.59265597319467, 83.42625869088596 27.59296979252196)
MULTIPOINT (83.42559174915993 27.59265597319467, 83.42625869088596 27.59296979252196)
i: 683
end
POINT (83.4527008616555 27.57518962190358)
POINT (83.4527008616555 27.57518962190358)
i: 684
start
POINT (83.43297050308632 27.58761966541527)
POINT (83.43297050308632 27.58761966541527)
i: 685
end
L

POINT (83.43546587265195 27.581138773036)
POINT (83.43546587265195 27.581138773036)
i: 759
POINT (83.43602977905113 27.56800354412832)
POINT (83.43602977905113 27.56800354412832)
i: 760
POINT (83.44173838042195 27.57001953996985)
POINT (83.44173838042195 27.57001953996985)
i: 761
start
POINT (83.43142587340537 27.57241225821917)
POINT (83.43142587340537 27.57241225821917)
i: 762
end
POINT (83.43031848870737 27.56010152596468)
POINT (83.43031848870737 27.56010152596468)
i: 763
POINT (83.42102282174591 27.56561971297941)
POINT (83.42102282174591 27.56561971297941)
i: 764
POINT (83.44483490900393 27.58470312175172)
POINT (83.44483490900393 27.58470312175172)
i: 765
POINT (83.42879434108602 27.58610148839301)
POINT (83.42879434108602 27.58610148839301)
i: 766
POINT (83.42598189484863 27.58861521851759)
POINT (83.42598189484863 27.58861521851759)
i: 767
end
POINT (83.43591666502215 27.56715490787275)
POINT (83.43591666502215 27.56715490787275)
i: 768
POINT (83.44348229537167 27.572546357928

POINT (83.44989912904163 27.60673280847237)
POINT (83.44989912904163 27.60673280847237)
i: 846
end
POINT (83.44984647103786 27.60320032543473)
POINT (83.44984647103786 27.60320032543473)
i: 847
POINT (83.44967369058838 27.60685002610785)
POINT (83.44967369058838 27.60685002610785)
i: 848
end
POINT (83.45204043551774 27.60526345474472)
POINT (83.45204043551774 27.60526345474472)
i: 849
POINT (83.45140427129013 27.60601735371796)
POINT (83.45140427129013 27.60601735371796)
i: 850
POINT (83.44971529502493 27.60273257005213)
POINT (83.44971529502493 27.60273257005213)
i: 851
POINT (83.44914352405448 27.60248997343359)
POINT (83.44914352405448 27.60248997343359)
i: 852
POINT (83.44967369058838 27.60685002610785)
POINT (83.44967369058838 27.60685002610785)
i: 853
end
POINT (83.45151996367542 27.60629622089391)
POINT (83.45151996367542 27.60629622089391)
i: 854
POINT (83.44973302425973 27.6067419878525)
POINT (83.44973302425973 27.6067419878525)
i: 855
POINT (83.44964634395987 27.604914473734

In [24]:
def assign_building_code(index,row):  

    road_linked_wrt_point= road.loc[road['NEW'] == row.NEW]
#     print(road_linked_wrt_point)
    for l in road_linked_wrt_point.geometry:
        line_geom=l
        break

    my_transformer = Transformer.from_crs('EPSG:4326', 'EPSG:32644', always_xy=True)
    geom_transformed = ops.transform(my_transformer.transform, line_geom)

    road_length=geom_transformed.length

    clipped_length,splitted_mother_road_side,building_point_projected_on_road_single_geom,start_point,degree_length_for_building=get_building_projected_road_length(road_linked_wrt_point,row)

    splitted_with_buffer_boi=split(splitted_mother_road_side,building_point_projected_on_road_single_geom.buffer(0.0001)) 

    for r  in splitted_with_buffer_boi:
        roi_line=r
        roi_line.srid = 4326
        if roi_line.buffer(0.00001).contains(start_point.buffer(0.0000001)) == False:
            break
        else:
            continue
#     print(roi_line)
#     print(shapely.geometry.mapping(roi_line))
    df_line = roi_line.boundary
    start_roi,end_roi=df_line[0],df_line[1]
#     print(start_roi)
#     print(end_roi)
#     print(building_point_projected_on_road_single_geom)
    if (round(start_roi.x,6) == round(building_point_projected_on_road_single_geom.x,4)) and (round(start_roi.y,4) == round(building_point_projected_on_road_single_geom.y,4)) :
        start_roi=end_roi
        end_roi=building_point_projected_on_road_single_geom
        print("start_end changed")

    road_index=road_linked_wrt_point.index[0]
#         road.at[road_index,'length']=length[0]       

    poi = row.geometry
    #constructing new geometry with new start and end for checking left and right 
    direction_updated_roi=LineString([start_roi,end_roi])

    left_buffered_line=direction_updated_roi.buffer(0.01,single_sided=True)
#     right_buffered_line=direction_updated_roi.buffer(0.0007,single_sided=True)
    if left_buffered_line.intersects(poi.buffer(0.00004)) == True:
        direction=-1
#         print("left")
    else:
        direction=1
#         print("right")
        
#     print(shapely.geometry.mapping(left_buffered_line))
#     print(shapely.geometry.mapping(poi.buffer(0.00004)))
    
    # break
#     print(clipped_length)
#     print(direction)
    buildings.at[index,'distance2_intersecting_road']=clipped_length
    road.at[road_index,'length']=road_length
    
    
    addDirection_BuildingCode(direction,clipped_length,road_length,index,road_index)
    
        
#     print("added building code")


In [ ]:
i=6531
row=buildings.iloc[i]
assign_building_code(i,row)
print(buildings.iloc[i])

In [ ]:
row_fetched=0
max_loop=len(buildings.index)
while row_fetched<max_loop:
    start=row_fetched
    end = row_fetched+100
    for i in range(start,end):
        row=buildings.iloc[i]
        try:
            assign_building_code(i,row)
            print(buildings.iloc[i].road_length_d_code)         
        except:
            continue
        print((f"""i: {i}/ {max_loop}"""))
    print(f"""Batch Done, {end} / {max_loop} """)
    row_fetched=end

C:\Users\Kshitiz\AppData\Local\Temp/ipykernel_10824/830731369.py:332: UserWarning: Geometry is in a geographic CRS. Results from 'interpolate' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  building_point_projected_on_road=linked_road_geometry.interpolate(degree_length_for_building) #end_always


670
i: 0/ 6545
659
i: 1/ 6545
651
i: 2/ 6545
206
i: 3/ 6545
214
i: 4/ 6545
214
i: 5/ 6545
193
i: 6/ 6545
185
i: 7/ 6545
182
i: 8/ 6545
215
i: 9/ 6545
214
i: 10/ 6545
214/38
i: 11/ 6545
214/32
i: 12/ 6545
46/26
i: 13/ 6545
169
i: 14/ 6545
170
i: 15/ 6545
173
i: 16/ 6545
166
i: 17/ 6545
178
i: 18/ 6545
188
i: 19/ 6545
198
i: 20/ 6545
203
i: 21/ 6545
215
i: 22/ 6545
214
i: 23/ 6545
227
i: 24/ 6545
232
i: 25/ 6545
244
i: 26/ 6545
243
i: 27/ 6545
253
i: 28/ 6545
263
i: 29/ 6545
271
i: 30/ 6545
278
i: 31/ 6545
260
i: 32/ 6545
291
i: 33/ 6545
288
i: 34/ 6545
305
i: 35/ 6545
313
i: 36/ 6545
323
i: 37/ 6545
949
i: 38/ 6545
14
i: 39/ 6545
368
i: 40/ 6545
384
i: 41/ 6545
359
i: 42/ 6545
923
i: 43/ 6545
917
i: 44/ 6545
911
i: 45/ 6545
405
i: 46/ 6545
279
i: 47/ 6545
425
i: 48/ 6545
175
i: 49/ 6545
180
i: 50/ 6545
424
i: 51/ 6545
450
i: 52/ 6545
454
i: 53/ 6545
460
i: 54/ 6545
21
i: 55/ 6545
17
i: 56/ 6545
31
i: 57/ 6545
49
i: 58/ 6545
66
i: 59/ 6545
93
i: 60/ 6545
107
i: 61/ 6545
158
i: 62/ 6545
1

641
i: 487/ 6545
53/7
i: 488/ 6545
53/11
i: 489/ 6545
53/15
i: 490/ 6545
53/21
i: 491/ 6545
53/18
i: 492/ 6545
53/30
i: 493/ 6545
331/17
i: 498/ 6545
331/17
i: 499/ 6545
Batch Done, 500 / 6545 
371
i: 500/ 6545
361
i: 501/ 6545
355
i: 502/ 6545
353
i: 503/ 6545
341
i: 504/ 6545
347
i: 505/ 6545
338
i: 506/ 6545
759
i: 507/ 6545
756
i: 508/ 6545
63
i: 510/ 6545
843
i: 511/ 6545
836
i: 512/ 6545
864
i: 513/ 6545
174
i: 514/ 6545
242
i: 515/ 6545
256
i: 516/ 6545
268
i: 517/ 6545
292
i: 518/ 6545
248
i: 519/ 6545
269
i: 520/ 6545
277
i: 521/ 6545
278
i: 522/ 6545
12
i: 523/ 6545
27
i: 524/ 6545
256
i: 525/ 6545
377
i: 526/ 6545
82/10
i: 527/ 6545
119
i: 529/ 6545
120
i: 530/ 6545
67
i: 531/ 6545
41
i: 532/ 6545
114
i: 533/ 6545
410
i: 534/ 6545
394
i: 535/ 6545
384
i: 536/ 6545
372
i: 537/ 6545
364
i: 538/ 6545
356
i: 539/ 6545
42
i: 540/ 6545
84
i: 541/ 6545
94
i: 542/ 6545
102
i: 543/ 6545
114
i: 544/ 6545
120
i: 545/ 6545
40
i: 546/ 6545
3230
i: 547/ 6545
331
i: 548/ 6545
130
i: 549/ 6

69
i: 953/ 6545
14
i: 954/ 6545
1092
i: 955/ 6545
1102
i: 956/ 6545
522
i: 957/ 6545
514
i: 958/ 6545
506
i: 959/ 6545
500
i: 960/ 6545
1078
i: 961/ 6545
1071
i: 962/ 6545
1061
i: 963/ 6545
550
i: 964/ 6545
558
i: 965/ 6545
570
i: 966/ 6545
578
i: 967/ 6545
580
i: 968/ 6545
585
i: 969/ 6545
591
i: 970/ 6545
612
i: 971/ 6545
625
i: 972/ 6545
629
i: 973/ 6545
639
i: 974/ 6545
630
i: 975/ 6545
650
i: 976/ 6545
654
i: 977/ 6545
657
i: 978/ 6545
667
i: 979/ 6545
642
i: 980/ 6545
642
i: 981/ 6545
622
i: 982/ 6545
666
i: 983/ 6545
672
i: 984/ 6545
682
i: 985/ 6545
684
i: 986/ 6545
690
i: 987/ 6545
704
i: 988/ 6545
716
i: 989/ 6545
701
i: 990/ 6545
709
i: 991/ 6545
721
i: 992/ 6545
727
i: 993/ 6545
733
i: 994/ 6545
739
i: 995/ 6545
757
i: 996/ 6545
746
i: 997/ 6545
762
i: 998/ 6545
783
i: 999/ 6545
Batch Done, 1000 / 6545 
257
i: 1000/ 6545
752
i: 1001/ 6545
752
i: 1002/ 6545
805
i: 1003/ 6545
816
i: 1004/ 6545
861
i: 1005/ 6545
872
i: 1006/ 6545
867
i: 1007/ 6545
148
i: 1008/ 6545
152
i: 1009

37
i: 1403/ 6545
51
i: 1404/ 6545
142
i: 1405/ 6545
347
i: 1406/ 6545
355
i: 1407/ 6545
345
i: 1408/ 6545
290
i: 1409/ 6545
135
i: 1410/ 6545
139
i: 1411/ 6545
157
i: 1412/ 6545
159
i: 1413/ 6545
94
i: 1414/ 6545
241
i: 1415/ 6545
132
i: 1416/ 6545
130
i: 1417/ 6545
149
i: 1418/ 6545
482
i: 1419/ 6545
464
i: 1420/ 6545
454
i: 1421/ 6545
424
i: 1422/ 6545
420
i: 1423/ 6545
356
i: 1424/ 6545
348
i: 1425/ 6545
336
i: 1426/ 6545
322
i: 1427/ 6545
304
i: 1428/ 6545
308
i: 1429/ 6545
332
i: 1430/ 6545
324
i: 1431/ 6545
322
i: 1432/ 6545
290
i: 1433/ 6545
280
i: 1434/ 6545
272
i: 1435/ 6545
136
i: 1436/ 6545
128
i: 1437/ 6545
112
i: 1438/ 6545
100
i: 1439/ 6545
90
i: 1440/ 6545
56
i: 1441/ 6545
24
i: 1442/ 6545
661
i: 1443/ 6545
630
i: 1444/ 6545
638
i: 1445/ 6545
662
i: 1446/ 6545
674
i: 1447/ 6545
682
i: 1448/ 6545
687
i: 1449/ 6545
677
i: 1450/ 6545
10
i: 1451/ 6545
723
i: 1452/ 6545
732
i: 1453/ 6545
782
i: 1454/ 6545
806
i: 1455/ 6545
817
i: 1456/ 6545
677
i: 1457/ 6545
106
i: 1458/ 6545

...

2595
i: 4101/ 6545
7
i: 4102/ 6545
9
i: 4103/ 6545
11
i: 4104/ 6545
84
i: 4105/ 6545
90
i: 4106/ 6545
2737
i: 4107/ 6545
2384
i: 4108/ 6545
2365
i: 4109/ 6545
390
i: 4110/ 6545
352
i: 4111/ 6545
2318
i: 4112/ 6545
2240
i: 4113/ 6545
270
i: 4114/ 6545
246
i: 4115/ 6545
214
i: 4116/ 6545
234
i: 4117/ 6545
2200
i: 4118/ 6545
2154
i: 4119/ 6545
2146
i: 4120/ 6545
2106
i: 4121/ 6545
40
i: 4122/ 6545
17
i: 4123/ 6545
2010
i: 4124/ 6545
1992
i: 4125/ 6545
1980
i: 4126/ 6545
1964
i: 4127/ 6545
1957
i: 4128/ 6545
1904
i: 4129/ 6545
1872
i: 4130/ 6545
1915
i: 4131/ 6545
1840
i: 4132/ 6545
1812
i: 4133/ 6545
1784
i: 4134/ 6545
1766
i: 4135/ 6545
1756
i: 4136/ 6545
1746
i: 4137/ 6545
20
i: 4138/ 6545
195
i: 4139/ 6545
161
i: 4140/ 6545
169
i: 4141/ 6545
179
i: 4142/ 6545
172
i: 4143/ 6545
168
i: 4144/ 6545
191
i: 4145/ 6545
195
i: 4146/ 6545
195
i: 4147/ 6545
201
i: 4148/ 6545
193
i: 4149/ 6545
217
i: 4150/ 6545
217
i: 4151/ 6545
215
i: 4152/ 6545
223
i: 4153/ 6545
241
i: 4154/ 6545
243
i: 4155/ 6

487
i: 4544/ 6545
434
i: 4545/ 6545
448
i: 4546/ 6545
558
i: 4547/ 6545
1573
i: 4548/ 6545
682
i: 4549/ 6545
1584
i: 4550/ 6545
714
i: 4551/ 6545
743
i: 4552/ 6545
761
i: 4553/ 6545
789
i: 4554/ 6545
1570
i: 4555/ 6545
1560
i: 4556/ 6545
1546
i: 4557/ 6545
1536
i: 4558/ 6545
1521
i: 4559/ 6545
1501
i: 4560/ 6545
1506
i: 4561/ 6545
1538
i: 4562/ 6545
1554
i: 4563/ 6545
734
i: 4564/ 6545
68
i: 4565/ 6545
34
i: 4566/ 6545
36
i: 4567/ 6545
27
i: 4568/ 6545
19
i: 4569/ 6545
124
i: 4570/ 6545
134
i: 4571/ 6545
1437
i: 4574/ 6545
1448
i: 4575/ 6545
1450
i: 4576/ 6545
1432
i: 4577/ 6545
1420
i: 4578/ 6545
1422
i: 4579/ 6545
1452
i: 4580/ 6545
1398
i: 4581/ 6545
1380
i: 4582/ 6545
1372
i: 4583/ 6545
1351
i: 4584/ 6545
1343
i: 4585/ 6545
1338
i: 4586/ 6545
1354
i: 4587/ 6545
1360
i: 4588/ 6545
1362
i: 4589/ 6545
1364
i: 4590/ 6545
1368
i: 4591/ 6545
414
i: 4592/ 6545
14
i: 4593/ 6545
13
i: 4594/ 6545
35
i: 4595/ 6545
440
i: 4596/ 6545
299
i: 4597/ 6545
297
i: 4598/ 6545
298
i: 4599/ 6545
Batch D

1149/13
i: 4987/ 6545
1149/11
i: 4988/ 6545
43
i: 4989/ 6545
1149/10
i: 4990/ 6545
71
i: 4991/ 6545
1133/0
i: 4992/ 6545
1149/20
i: 4993/ 6545
920
i: 4994/ 6545
942
i: 4995/ 6545
952
i: 4996/ 6545
61
i: 4997/ 6545
85
i: 4998/ 6545
73
i: 4999/ 6545
Batch Done, 5000 / 6545 
277
i: 5000/ 6545
286
i: 5001/ 6545
286
i: 5002/ 6545
299
i: 5003/ 6545
435
i: 5004/ 6545
2763
i: 5005/ 6545
2769
i: 5006/ 6545
2792
i: 5007/ 6545
2780
i: 5008/ 6545
135
i: 5009/ 6545
5
i: 5010/ 6545
19
i: 5011/ 6545
21
i: 5012/ 6545
29
i: 5013/ 6545
43
i: 5014/ 6545
44
i: 5015/ 6545
26/10
i: 5016/ 6545
101
i: 5017/ 6545
26/16
i: 5018/ 6545
26/12
i: 5019/ 6545
26/9
i: 5020/ 6545
26/17
i: 5021/ 6545
74/10
i: 5022/ 6545
77
i: 5023/ 6545
74/14
i: 5024/ 6545
74/21
i: 5025/ 6545
76
i: 5026/ 6545
72
i: 5027/ 6545
45
i: 5028/ 6545
47
i: 5029/ 6545
41
i: 5030/ 6545
29
i: 5031/ 6545
37
i: 5032/ 6545
23
i: 5033/ 6545
17
i: 5034/ 6545
65
i: 5035/ 6545
50
i: 5036/ 6545
29
i: 5037/ 6545
57
i: 5038/ 6545
49
i: 5039/ 6545
61
i: 5040

44
i: 5438/ 6545
56
i: 5439/ 6545
55
i: 5440/ 6545
54
i: 5441/ 6545
82
i: 5442/ 6545
86
i: 5443/ 6545
94
i: 5444/ 6545
92
i: 5445/ 6545
101
i: 5446/ 6545
95
i: 5447/ 6545
114
i: 5448/ 6545
121
i: 5449/ 6545
123
i: 5450/ 6545
127
i: 5451/ 6545
136
i: 5452/ 6545
137
i: 5453/ 6545
131
i: 5454/ 6545
112
i: 5455/ 6545
110
i: 5456/ 6545
118
i: 5457/ 6545
134
i: 5458/ 6545
33
i: 5459/ 6545
45
i: 5460/ 6545
145
i: 5461/ 6545
151
i: 5462/ 6545
155
i: 5463/ 6545
165
i: 5464/ 6545
65
i: 5465/ 6545
65
i: 5466/ 6545
159
i: 5467/ 6545
168
i: 5468/ 6545
162
i: 5469/ 6545
944
i: 5470/ 6545
941
i: 5471/ 6545
921
i: 5472/ 6545
915
i: 5473/ 6545
907
i: 5474/ 6545
899
i: 5475/ 6545
893
i: 5476/ 6545
867
i: 5477/ 6545
861
i: 5478/ 6545
845
i: 5479/ 6545
850
i: 5480/ 6545
842
i: 5481/ 6545
872
i: 5482/ 6545
860
i: 5483/ 6545
848
i: 5484/ 6545
808
i: 5485/ 6545
803
i: 5486/ 6545
780
i: 5487/ 6545
774
i: 5488/ 6545
768
i: 5489/ 6545
740
i: 5490/ 6545
9
i: 5491/ 6545
974
i: 5492/ 6545
714
i: 5493/ 6545
706
i: 

16
i: 5884/ 6545
757
i: 5885/ 6545
708
i: 5886/ 6545
709
i: 5887/ 6545
583
i: 5888/ 6545
584
i: 5889/ 6545
529
i: 5890/ 6545
64
i: 5891/ 6545
187
i: 5892/ 6545
233
i: 5893/ 6545
259
i: 5894/ 6545
108
i: 5895/ 6545
86
i: 5896/ 6545
301
i: 5897/ 6545
582/36
i: 5898/ 6545
1011
i: 5899/ 6545
Batch Done, 5900 / 6545 
1069
i: 5900/ 6545
1079
i: 5901/ 6545
1068
i: 5902/ 6545
1094
i: 5903/ 6545
1103
i: 5904/ 6545
1131
i: 5905/ 6545
1148
i: 5906/ 6545
1134
i: 5907/ 6545
1114
i: 5908/ 6545
1124
i: 5909/ 6545
233
i: 5910/ 6545
253
i: 5911/ 6545
259
i: 5912/ 6545
558
i: 5913/ 6545
768
i: 5914/ 6545
770
i: 5915/ 6545
19
i: 5916/ 6545
10
i: 5917/ 6545
103
i: 5918/ 6545
95
i: 5919/ 6545
89
i: 5920/ 6545
88
i: 5921/ 6545
308
i: 5922/ 6545
314
i: 5923/ 6545
354
i: 5924/ 6545
372
i: 5925/ 6545
392
i: 5926/ 6545
408
i: 5927/ 6545
416
i: 5928/ 6545
437
i: 5929/ 6545
52
i: 5930/ 6545
128
i: 5931/ 6545
65
i: 5932/ 6545
9
i: 5933/ 6545
430
i: 5934/ 6545
722
i: 5935/ 6545
799
i: 5936/ 6545
780
i: 5937/ 6545
7

In [ ]:
buildings.to_file("output_building_1.geojson", driver="GeoJSON")

In [ ]:
print(buildings)

In [ ]:
road.to_file("output_road_1.geojson", driver="GeoJSON")